In [101]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))#to expand the size of my container so it can take fill up my entire monitor

In [102]:
import pandas as pd
import numpy as np

import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, LSTM, GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dropout, Flatten, Conv1D, MaxPooling1D


from keras import layers
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [103]:
train = pd.read_csv('../data/train.csv')
train.head()

,sequence,is_promoter
0,TTAATTTGTCCTTATTTGATTAAGAAGAATAAATCTTATATATAGA...,1
1,ATAGCTCAAATTGCTTTATTAGTATTAGAATCAGCTGTAGCTATAA...,1
2,AAGCTTCCCTTTAATGTGCTCCTTGTGAATACAGCATTACAATGCC...,1
3,TATGTAGAATCTGTACAAGTATCTGTGTTTGGACAATGGCATGTGT...,1
4,ACATATTACTGCATACAGGTCTCAAATTATAAAATGACACTCGTGG...,1


In [104]:
# k-mer counter function
def two_mers_funct(seq, size=2):
    return [seq[x:x+size].lower() for x in range(len(seq) - size + 1)]

In [105]:
def three_mers_funct(seq, size=3):
    return [seq[x:x+size].lower() for x in range(len(seq) - size + 1)]

In [106]:
def four_mers_funct(seq, size=4):
    return [seq[x:x+size].lower() for x in range(len(seq) - size + 1)]

In [107]:
def five_mers_funct(seq, size=5):
    return [seq[x:x+size].lower() for x in range(len(seq) - size + 1)]

In [108]:
def six_mers_funct(seq, size=6):
    return [seq[x:x+size].lower() for x in range(len(seq) - size + 1)]

In [109]:
train['2_mer'] = train['sequence'].apply(two_mers_funct)
train['3_mer'] = train['sequence'].apply(three_mers_funct)
train['4_mer'] = train['sequence'].apply(four_mers_funct)
train['5_mer'] = train['sequence'].apply(five_mers_funct)
train['6_mer'] = train['sequence'].apply(six_mers_funct)

In [110]:
train.head(2)

,sequence,is_promoter,2_mer,3_mer,4_mer,5_mer,6_mer
0,TTAATTTGTCCTTATTTGATTAAGAAGAATAAATCTTATATATAGA...,1,"[tt, ta, aa, at, tt, tt, tg, gt, tc, cc, ct, t...","[tta, taa, aat, att, ttt, ttg, tgt, gtc, tcc, ...","[ttaa, taat, aatt, attt, tttg, ttgt, tgtc, gtc...","[ttaat, taatt, aattt, atttg, tttgt, ttgtc, tgt...","[ttaatt, taattt, aatttg, atttgt, tttgtc, ttgtc..."
1,ATAGCTCAAATTGCTTTATTAGTATTAGAATCAGCTGTAGCTATAA...,1,"[at, ta, ag, gc, ct, tc, ca, aa, aa, at, tt, t...","[ata, tag, agc, gct, ctc, tca, caa, aaa, aat, ...","[atag, tagc, agct, gctc, ctca, tcaa, caaa, aaa...","[atagc, tagct, agctc, gctca, ctcaa, tcaaa, caa...","[atagct, tagctc, agctca, gctcaa, ctcaaa, tcaaa..."


In [111]:
train.drop(train.index[[13122]], inplace=True)

In [112]:
train['2_mer'] = train['2_mer'].apply(" ".join)
train['3_mer'] = train['3_mer'].apply(" ".join)
train['4_mer'] = train['4_mer'].apply(" ".join)
train['5_mer'] = train['5_mer'].apply(" ".join)
train['6_mer'] = train['6_mer'].apply(" ".join)
train.head()

,sequence,is_promoter,2_mer,3_mer,4_mer,5_mer,6_mer
0,TTAATTTGTCCTTATTTGATTAAGAAGAATAAATCTTATATATAGA...,1,tt ta aa at tt tt tg gt tc cc ct tt ta at tt t...,tta taa aat att ttt ttg tgt gtc tcc cct ctt tt...,ttaa taat aatt attt tttg ttgt tgtc gtcc tcct c...,ttaat taatt aattt atttg tttgt ttgtc tgtcc gtcc...,ttaatt taattt aatttg atttgt tttgtc ttgtcc tgtc...
1,ATAGCTCAAATTGCTTTATTAGTATTAGAATCAGCTGTAGCTATAA...,1,at ta ag gc ct tc ca aa aa at tt tg gc ct tt t...,ata tag agc gct ctc tca caa aaa aat att ttg tg...,atag tagc agct gctc ctca tcaa caaa aaat aatt a...,atagc tagct agctc gctca ctcaa tcaaa caaat aaat...,atagct tagctc agctca gctcaa ctcaaa tcaaat caaa...
2,AAGCTTCCCTTTAATGTGCTCCTTGTGAATACAGCATTACAATGCC...,1,aa ag gc ct tt tc cc cc ct tt tt ta aa at tg g...,aag agc gct ctt ttc tcc ccc cct ctt ttt tta ta...,aagc agct gctt cttc ttcc tccc ccct cctt cttt t...,aagct agctt gcttc cttcc ttccc tccct ccctt cctt...,aagctt agcttc gcttcc cttccc ttccct tccctt ccct...
3,TATGTAGAATCTGTACAAGTATCTGTGTTTGGACAATGGCATGTGT...,1,ta at tg gt ta ag ga aa at tc ct tg gt ta ac c...,tat atg tgt gta tag aga gaa aat atc tct ctg tg...,tatg atgt tgta gtag taga agaa gaat aatc atct t...,tatgt atgta tgtag gtaga tagaa agaat gaatc aatc...,tatgta atgtag tgtaga gtagaa tagaat agaatc gaat...
4,ACATATTACTGCATACAGGTCTCAAATTATAAAATGACACTCGTGG...,1,ac ca at ta at tt ta ac ct tg gc ca at ta ac c...,aca cat ata tat att tta tac act ctg tgc gca ca...,acat cata atat tatt atta ttac tact actg ctgc t...,acata catat atatt tatta attac ttact tactg actg...,acatat catatt atatta tattac attact ttactg tact...


In [113]:
from keras import layers

In [114]:
import matplotlib.pyplot as plt

In [115]:
from tensorflow.keras import regularizers

# 2-mer

In [116]:
X = train['2_mer']
y = train['is_promoter']

In [117]:
# Split train & test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tokenize and transform to integer index
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index
maxlen = max(len(x) for x in X_train) # longest text in train set

# Add pading to ensure all vectors have same dimensionality
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)


In [118]:
embedding_dim = 200

model_2 = Sequential()
model_2.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
model_2.add(Conv1D(200, 5, activation='relu'))
model_2.add(MaxPooling1D(pool_size=2))
model_2.add(layers.Conv1D(200, 5, activation='relu'))
model_2.add(MaxPooling1D(pool_size=2))
model_2.add(Flatten())
model_2.add(Dense(128, activation = 'relu'))
model_2.add(Dense(32, activation = 'relu'))
model_2.add(Dense(10, activation='relu'))
model_2.add(Dense(1, activation='sigmoid'))
model_2.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
print(model_2.summary())

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 300, 200)          3400      
                                                                 
 conv1d_14 (Conv1D)          (None, 296, 200)          200200    
                                                                 
 max_pooling1d_14 (MaxPoolin  (None, 148, 200)         0         
 g1D)                                                            
                                                                 
 conv1d_15 (Conv1D)          (None, 144, 200)          200200    
                                                                 
 max_pooling1d_15 (MaxPoolin  (None, 72, 200)          0         
 g1D)                                                            
                                                                 
 flatten_7 (Flatten)         (None, 14400)            

In [119]:
# # create callback
# filepath = 'my_best_model_2'
# checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=filepath, 
#                              monitor='val_loss',
#                              verbose=1, 
#                              save_best_only=True,
#                              mode='min')
# callbacks = [checkpoint]

In [120]:
# # saving best model
# checkpoint_filepath = '../my_best_model_2.epoch{epoch:02d}-loss{val_loss:.2f}.hdf5'
# model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
#     filepath=checkpoint_filepath,
#     save_weights_only=True,
#     monitor='val_loss',
#     verbose=1,
#     mode='min',
#     save_best_only=True)
# callbacks = [model_checkpoint_callback]
# this didn't work...

In [121]:
history = model_2.fit(X_train,
                    y_train, 
                    batch_size=256,
                    validation_data=(X_test, y_test),
                    epochs=10,
                    callbacks=callbacks,
                    verbose=1)

Epoch 1/10
71/71 [==============================] - ETA: 0s - loss: 0.4790 - accuracy: 0.7662
Epoch 00001: val_loss improved from inf to 0.35528, saving model to ../my_best_model_2.epoch01-loss0.36.hdf5
71/71 [==============================] - 189s 3s/step - loss: 0.4790 - accuracy: 0.7662 - val_loss: 0.3553 - val_accuracy: 0.8476
Epoch 2/10
71/71 [==============================] - ETA: 0s - loss: 0.2987 - accuracy: 0.8741
Epoch 00002: val_loss improved from 0.35528 to 0.29247, saving model to ../my_best_model_2.epoch02-loss0.29.hdf5
71/71 [==============================] - 199s 3s/step - loss: 0.2987 - accuracy: 0.8741 - val_loss: 0.2925 - val_accuracy: 0.8757
Epoch 3/10
71/71 [==============================] - ETA: 0s - loss: 0.2346 - accuracy: 0.9071
Epoch 00003: val_loss improved from 0.29247 to 0.25213, saving model to ../my_best_model_2.epoch03-loss0.25.hdf5
71/71 [==============================] - 202s 3s/step - loss: 0.2346 - accuracy: 0.9071 - val_loss: 0.2521 - val_accuracy: 

In [123]:
# best - loss: 0.2346 - accuracy: 0.9071 - val_loss: 0.2521 - val_accuracy: 0.8967

# 3-mer

In [125]:
X = train['3_mer']
y = train['is_promoter']

In [126]:
# Split train & test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tokenize and transform to integer index
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index
maxlen = max(len(x) for x in X_train) # longest text in train set

# Add pading to ensure all vectors have same dimensionality
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)


In [127]:
embedding_dim = 200

model_3 = Sequential()
model_3.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
model_3.add(Conv1D(200, 5, activation='relu'))
model_3.add(MaxPooling1D(pool_size=2))
model_3.add(layers.Conv1D(200, 5, activation='relu'))
model_3.add(MaxPooling1D(pool_size=2))
model_3.add(Flatten())
model_3.add(Dense(128, activation = 'relu'))
model_3.add(Dense(32, activation = 'relu'))
model_3.add(Dense(10, activation='relu'))
model_3.add(Dense(1, activation='sigmoid'))
model_3.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
print(model_3.summary())

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, 299, 200)          13000     
                                                                 
 conv1d_16 (Conv1D)          (None, 295, 200)          200200    
                                                                 
 max_pooling1d_16 (MaxPoolin  (None, 147, 200)         0         
 g1D)                                                            
                                                                 
 conv1d_17 (Conv1D)          (None, 143, 200)          200200    
                                                                 
 max_pooling1d_17 (MaxPoolin  (None, 71, 200)          0         
 g1D)                                                            
                                                                 
 flatten_8 (Flatten)         (None, 14200)            

In [130]:
history = model_3.fit(X_train,
                    y_train, 
                    batch_size=256,
                    validation_data=(X_test, y_test),
                    epochs=10,
                    callbacks=callbacks, 
                    verbose=1)

Epoch 1/10
71/71 [==============================] - ETA: 0s - loss: 0.5188 - accuracy: 0.7279
Epoch 00001: val_loss improved from inf to 0.41618, saving model to ../my_best_model_3.epoch01-loss0.42.hdf5
71/71 [==============================] - 198s 3s/step - loss: 0.5188 - accuracy: 0.7279 - val_loss: 0.4162 - val_accuracy: 0.8148
Epoch 2/10
71/71 [==============================] - ETA: 0s - loss: 0.3379 - accuracy: 0.8517
Epoch 00002: val_loss improved from 0.41618 to 0.29989, saving model to ../my_best_model_3.epoch02-loss0.30.hdf5
71/71 [==============================] - 162s 2s/step - loss: 0.3379 - accuracy: 0.8517 - val_loss: 0.2999 - val_accuracy: 0.8746
Epoch 3/10
71/71 [==============================] - ETA: 0s - loss: 0.2537 - accuracy: 0.8958
Epoch 00003: val_loss did not improve from 0.29989
71/71 [==============================] - 158s 2s/step - loss: 0.2537 - accuracy: 0.8958 - val_loss: 0.3269 - val_accuracy: 0.8741
Epoch 4/10
71/71 [==============================] - ETA

In [131]:
# best - loss: 0.1694 - accuracy: 0.9355 - val_loss: 0.2647 - val_accuracy: 0.8960
# best acc - loss: 0.1290 - accuracy: 0.9517 - val_loss: 0.2806 - val_accuracy: 0.8969

# 4-mer

In [132]:
X = train['4_mer']
y = train['is_promoter']

In [133]:
# Split train & test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tokenize and transform to integer index
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index
maxlen = max(len(x) for x in X_train) # longest text in train set

# Add pading to ensure all vectors have same dimensionality
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)


In [134]:
embedding_dim = 200

model_4 = Sequential()
model_4.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
model_4.add(Conv1D(200, 5, activation='relu'))
model_4.add(MaxPooling1D(pool_size=2))
model_4.add(layers.Conv1D(200, 5, activation='relu'))
model_4.add(MaxPooling1D(pool_size=2))
model_4.add(Flatten())
model_4.add(Dense(128, activation = 'relu'))
model_4.add(Dense(32, activation = 'relu'))
model_4.add(Dense(10, activation='relu'))
model_4.add(Dense(1, activation='sigmoid'))
model_4.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
print(model_4.summary())

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     (None, 298, 200)          51400     
                                                                 
 conv1d_18 (Conv1D)          (None, 294, 200)          200200    
                                                                 
 max_pooling1d_18 (MaxPoolin  (None, 147, 200)         0         
 g1D)                                                            
                                                                 
 conv1d_19 (Conv1D)          (None, 143, 200)          200200    
                                                                 
 max_pooling1d_19 (MaxPoolin  (None, 71, 200)          0         
 g1D)                                                            
                                                                 
 flatten_9 (Flatten)         (None, 14200)            

In [137]:
history = model_4.fit(X_train,
                    y_train, 
                    batch_size=256,
                    validation_data=(X_test, y_test),
                    epochs=10,
                    callbacks=callbacks,
                    verbose=1)

Epoch 1/10
71/71 [==============================] - ETA: 0s - loss: 0.5058 - accuracy: 0.7519
Epoch 00001: val_loss improved from inf to 0.40716, saving model to ../my_best_model_4.epoch01-loss0.41.hdf5
71/71 [==============================] - 161s 2s/step - loss: 0.5058 - accuracy: 0.7519 - val_loss: 0.4072 - val_accuracy: 0.8186
Epoch 2/10
71/71 [==============================] - ETA: 0s - loss: 0.3205 - accuracy: 0.8636
Epoch 00002: val_loss improved from 0.40716 to 0.29346, saving model to ../my_best_model_4.epoch02-loss0.29.hdf5
71/71 [==============================] - 160s 2s/step - loss: 0.3205 - accuracy: 0.8636 - val_loss: 0.2935 - val_accuracy: 0.8768
Epoch 3/10
71/71 [==============================] - ETA: 0s - loss: 0.2393 - accuracy: 0.9037
Epoch 00003: val_loss improved from 0.29346 to 0.26688, saving model to ../my_best_model_4.epoch03-loss0.27.hdf5
71/71 [==============================] - 159s 2s/step - loss: 0.2393 - accuracy: 0.9037 - val_loss: 0.2669 - val_accuracy: 

In [138]:
# best score - loss: 0.1949 - accuracy: 0.9257 - val_loss: 0.2549 - val_accuracy: 0.9002

# 5-mer

In [197]:
X = train['5_mer']
y = train['is_promoter']

In [198]:
# Split train & test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tokenize and transform to integer index
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index
maxlen = max(len(x) for x in X_train) # longest text in train set

# Add pading to ensure all vectors have same dimensionality
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)


In [199]:
X_test.shape

(4520, 297)

In [141]:
embedding_dim = 200

model_5 = Sequential()
model_5.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
model_5.add(Conv1D(200, 5, activation='relu'))
model_5.add(MaxPooling1D(pool_size=2))
model_5.add(layers.Conv1D(200, 5, activation='relu'))
model_5.add(MaxPooling1D(pool_size=2))
model_5.add(Flatten())
model_5.add(Dense(128, activation = 'relu'))
model_5.add(Dense(32, activation = 'relu'))
model_5.add(Dense(10, activation='relu'))
model_5.add(Dense(1, activation='sigmoid'))
model_5.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
print(model_5.summary())

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_10 (Embedding)    (None, 297, 200)          205000    
                                                                 
 conv1d_20 (Conv1D)          (None, 293, 200)          200200    
                                                                 
 max_pooling1d_20 (MaxPoolin  (None, 146, 200)         0         
 g1D)                                                            
                                                                 
 conv1d_21 (Conv1D)          (None, 142, 200)          200200    
                                                                 
 max_pooling1d_21 (MaxPoolin  (None, 71, 200)          0         
 g1D)                                                            
                                                                 
 flatten_10 (Flatten)        (None, 14200)           

In [144]:
history = model_5.fit(X_train,
                    y_train, 
                    batch_size=256,
                    validation_data=(X_test, y_test),
                    epochs=10,
                    callbacks=callbacks,
                    verbose=1)

Epoch 1/10
71/71 [==============================] - ETA: 0s - loss: 0.5234 - accuracy: 0.7209
Epoch 00001: val_loss improved from inf to 0.40640, saving model to ../my_best_model_5.epoch01-loss0.41.hdf5
71/71 [==============================] - 155s 2s/step - loss: 0.5234 - accuracy: 0.7209 - val_loss: 0.4064 - val_accuracy: 0.8197
Epoch 2/10
71/71 [==============================] - ETA: 0s - loss: 0.3315 - accuracy: 0.8573
Epoch 00002: val_loss improved from 0.40640 to 0.31593, saving model to ../my_best_model_5.epoch02-loss0.32.hdf5
71/71 [==============================] - 153s 2s/step - loss: 0.3315 - accuracy: 0.8573 - val_loss: 0.3159 - val_accuracy: 0.8650
Epoch 3/10
71/71 [==============================] - ETA: 0s - loss: 0.2393 - accuracy: 0.9027
Epoch 00003: val_loss improved from 0.31593 to 0.30862, saving model to ../my_best_model_5.epoch03-loss0.31.hdf5
71/71 [==============================] - 158s 2s/step - loss: 0.2393 - accuracy: 0.9027 - val_loss: 0.3086 - val_accuracy: 

In [145]:
# best score: 

# 6-mer

In [146]:
X = train['6_mer']
y = train['is_promoter']

In [147]:
# Split train & test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tokenize and transform to integer index
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index
maxlen = max(len(x) for x in X_train) # longest text in train set

# Add pading to ensure all vectors have same dimensionality
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)


In [148]:
embedding_dim = 200

model_6 = Sequential()
model_6.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
model_6.add(Conv1D(200, 5, activation='relu'))
model_6.add(MaxPooling1D(pool_size=2))
model_6.add(layers.Conv1D(200, 5, activation='relu'))
model_6.add(MaxPooling1D(pool_size=2))
model_6.add(Flatten())
model_6.add(Dense(128, activation = 'relu'))
model_6.add(Dense(32, activation = 'relu'))
model_6.add(Dense(10, activation='relu'))
model_6.add(Dense(1, activation='sigmoid'))
model_6.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
print(model_6.summary())

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_11 (Embedding)    (None, 296, 200)          819400    
                                                                 
 conv1d_22 (Conv1D)          (None, 292, 200)          200200    
                                                                 
 max_pooling1d_22 (MaxPoolin  (None, 146, 200)         0         
 g1D)                                                            
                                                                 
 conv1d_23 (Conv1D)          (None, 142, 200)          200200    
                                                                 
 max_pooling1d_23 (MaxPoolin  (None, 71, 200)          0         
 g1D)                                                            
                                                                 
 flatten_11 (Flatten)        (None, 14200)           

In [151]:
history = model_6.fit(X_train,
                    y_train, 
                    batch_size=256,
                    validation_data=(X_test, y_test),
                    epochs=10,
                    callbacks=callbacks,
                    verbose=1)

Epoch 1/10
71/71 [==============================] - ETA: 0s - loss: 0.5977 - accuracy: 0.6474
Epoch 00001: val_loss improved from inf to 0.42324, saving model to ../my_best_model_6.epoch01-loss0.42.hdf5
71/71 [==============================] - 213s 3s/step - loss: 0.5977 - accuracy: 0.6474 - val_loss: 0.4232 - val_accuracy: 0.8084
Epoch 2/10
71/71 [==============================] - ETA: 0s - loss: 0.3499 - accuracy: 0.8487
Epoch 00002: val_loss improved from 0.42324 to 0.34006, saving model to ../my_best_model_6.epoch02-loss0.34.hdf5
71/71 [==============================] - 198s 3s/step - loss: 0.3499 - accuracy: 0.8487 - val_loss: 0.3401 - val_accuracy: 0.8533
Epoch 3/10
71/71 [==============================] - ETA: 0s - loss: 0.2439 - accuracy: 0.9028
Epoch 00003: val_loss did not improve from 0.34006
71/71 [==============================] - 201s 3s/step - loss: 0.2439 - accuracy: 0.9028 - val_loss: 0.3408 - val_accuracy: 0.8597
Epoch 4/10
71/71 [==============================] - ETA

In [152]:
# best score: 

# TEST DATA

In [153]:
test = pd.read_csv('../data/test.csv')
test.head()

,Sequence
0,AATCTCTGTTGTTGTTATGCAAAAAAGGAATAATGTAATGTATTAT...
1,TTCTCTAAGAAAGTCGATGCTAAGCGGATGCTAAACGCAGGGCCGG...
2,ACATTTTACGGTCTTGCGTGTGTATTTATTGTTCGTACAAGTCAAA...
3,CACTGAAAAAAAAAGAAAGGCTTATTTACTATTAACAACTTTAATA...
4,GGGAGTGGCAACATGGGCTCACAAGTCTAGATCGACTCGGCTATTG...


In [155]:
test['2_mer'] = test['Sequence'].apply(two_mers_funct)
test['4_mer'] = test['Sequence'].apply(four_mers_funct)
test.head()

,Sequence,2_mer,4_mer
0,AATCTCTGTTGTTGTTATGCAAAAAAGGAATAATGTAATGTATTAT...,"[aa, at, tc, ct, tc, ct, tg, gt, tt, tg, gt, t...","[aatc, atct, tctc, ctct, tctg, ctgt, tgtt, gtt..."
1,TTCTCTAAGAAAGTCGATGCTAAGCGGATGCTAAACGCAGGGCCGG...,"[tt, tc, ct, tc, ct, ta, aa, ag, ga, aa, aa, a...","[ttct, tctc, ctct, tcta, ctaa, taag, aaga, aga..."
2,ACATTTTACGGTCTTGCGTGTGTATTTATTGTTCGTACAAGTCAAA...,"[ac, ca, at, tt, tt, tt, ta, ac, cg, gg, gt, t...","[acat, catt, attt, tttt, ttta, ttac, tacg, acg..."
3,CACTGAAAAAAAAAGAAAGGCTTATTTACTATTAACAACTTTAATA...,"[ca, ac, ct, tg, ga, aa, aa, aa, aa, aa, aa, a...","[cact, actg, ctga, tgaa, gaaa, aaaa, aaaa, aaa..."
4,GGGAGTGGCAACATGGGCTCACAAGTCTAGATCGACTCGGCTATTG...,"[gg, gg, ga, ag, gt, tg, gg, gc, ca, aa, ac, c...","[ggga, ggag, gagt, agtg, gtgg, tggc, ggca, gca..."


In [156]:
test['2_mer'] = test['2_mer'].apply(" ".join)
test['4_mer'] = test['4_mer'].apply(" ".join)
test.head()

,Sequence,2_mer,4_mer
0,AATCTCTGTTGTTGTTATGCAAAAAAGGAATAATGTAATGTATTAT...,aa at tc ct tc ct tg gt tt tg gt tt tg gt tt t...,aatc atct tctc ctct tctg ctgt tgtt gttg ttgt t...
1,TTCTCTAAGAAAGTCGATGCTAAGCGGATGCTAAACGCAGGGCCGG...,tt tc ct tc ct ta aa ag ga aa aa ag gt tc cg g...,ttct tctc ctct tcta ctaa taag aaga agaa gaaa a...
2,ACATTTTACGGTCTTGCGTGTGTATTTATTGTTCGTACAAGTCAAA...,ac ca at tt tt tt ta ac cg gg gt tc ct tt tg g...,acat catt attt tttt ttta ttac tacg acgg cggt g...
3,CACTGAAAAAAAAAGAAAGGCTTATTTACTATTAACAACTTTAATA...,ca ac ct tg ga aa aa aa aa aa aa aa aa ag ga a...,cact actg ctga tgaa gaaa aaaa aaaa aaaa aaaa a...
4,GGGAGTGGCAACATGGGCTCACAAGTCTAGATCGACTCGGCTATTG...,gg gg ga ag gt tg gg gc ca aa ac ca at tg gg g...,ggga ggag gagt agtg gtgg tggc ggca gcaa caac a...


# Running my models again because I didn't save them correctly...

# 2-mer take two

In [244]:
X = train['2_mer']
y = train['is_promoter']

# Split train & test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tokenize and transform to integer index
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index
maxlen = max(len(x) for x in X_train) # longest text in train set

# Add pading to ensure all vectors have same dimensionality
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

embedding_dim = 200

model_2 = Sequential()
model_2.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
model_2.add(Conv1D(200, 5, activation='relu'))
model_2.add(MaxPooling1D(pool_size=2))
model_2.add(layers.Conv1D(200, 5, activation='relu'))
model_2.add(MaxPooling1D(pool_size=2))
model_2.add(Flatten())
model_2.add(Dense(128, activation = 'relu'))
model_2.add(Dense(32, activation = 'relu'))
model_2.add(Dense(10, activation='relu'))
model_2.add(Dense(1, activation='sigmoid'))
model_2.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
print(model_2.summary())

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_15 (Embedding)    (None, 300, 200)          3400      
                                                                 
 conv1d_30 (Conv1D)          (None, 296, 200)          200200    
                                                                 
 max_pooling1d_30 (MaxPoolin  (None, 148, 200)         0         
 g1D)                                                            
                                                                 
 conv1d_31 (Conv1D)          (None, 144, 200)          200200    
                                                                 
 max_pooling1d_31 (MaxPoolin  (None, 72, 200)          0         
 g1D)                                                            
                                                                 
 flatten_15 (Flatten)        (None, 14400)           

In [245]:
history = model_2.fit(X_train,
                    y_train, 
                    batch_size=256,
                    validation_data=(X_test, y_test),
                    epochs=3,
                    callbacks=callbacks,
                    verbose=1)

Epoch 1/3
71/71 [==============================] - ETA: 0s - loss: 0.4832 - accuracy: 0.7530
Epoch 00001: val_loss did not improve from 0.28680
71/71 [==============================] - 179s 3s/step - loss: 0.4832 - accuracy: 0.7530 - val_loss: 0.3548 - val_accuracy: 0.8465
Epoch 2/3
71/71 [==============================] - ETA: 0s - loss: 0.3024 - accuracy: 0.8732
Epoch 00002: val_loss improved from 0.28680 to 0.27745, saving model to ../my_best_model_6.epoch02-loss0.28.hdf5
71/71 [==============================] - 178s 3s/step - loss: 0.3024 - accuracy: 0.8732 - val_loss: 0.2774 - val_accuracy: 0.8874
Epoch 3/3
71/71 [==============================] - ETA: 0s - loss: 0.2533 - accuracy: 0.8970
Epoch 00003: val_loss improved from 0.27745 to 0.26332, saving model to ../my_best_model_6.epoch03-loss0.26.hdf5
71/71 [==============================] - 168s 2s/step - loss: 0.2533 - accuracy: 0.8970 - val_loss: 0.2633 - val_accuracy: 0.8925


# 2-mer predictions

In [246]:
X_test_final = test['2_mer']

In [247]:
X_test_final.shape

(178,)

In [248]:
X_test_final = tokenizer.texts_to_sequences(X_test_final)

maxlen = 300 # longest text in train set

# Add pading to ensure all vectors have same dimensionality
X_test_final = pad_sequences(X_test_final, padding='post', maxlen=maxlen)

In [250]:
X_test_final.shape

(178, 300)

In [251]:
preds_2 = model_2.predict(X_test_final)

In [252]:
preds_2

array([[2.30133533e-02],
       [9.56044257e-01],
       [4.55608517e-01],
       [1.42914057e-03],
       [2.90380716e-02],
       [3.62625122e-01],
       [8.50050807e-01],
       [7.28498995e-01],
       [6.26659691e-02],
       [9.96516228e-01],
       [9.97570157e-01],
       [7.58167207e-02],
       [2.96129286e-02],
       [7.42640078e-01],
       [3.75598848e-01],
       [9.95413780e-01],
       [6.54122233e-03],
       [1.36673868e-01],
       [5.18369257e-01],
       [8.10955048e-01],
       [1.75146163e-02],
       [6.11055970e-01],
       [9.72112894e-01],
       [3.48948240e-02],
       [1.26416862e-01],
       [9.88227844e-01],
       [9.21273947e-01],
       [9.85605359e-01],
       [9.86740887e-01],
       [9.17989612e-01],
       [5.81485033e-02],
       [2.65088975e-02],
       [4.81728464e-01],
       [1.73267514e-01],
       [9.91550148e-01],
       [8.28450918e-03],
       [1.44262016e-01],
       [2.79943764e-01],
       [3.93009782e-02],
       [5.03809690e-01],


In [253]:
labels=np.array(preds_2)
labels[labels>=0.5]=1
labels[labels<0.5]=0   

In [256]:
preds_2 = pd.DataFrame(data = labels, columns = ['predictions'])
preds_2

,predictions
0,0.0
1,1.0
2,0.0
3,0.0
4,0.0
...,...
173,1.0
174,1.0
175,0.0
176,0.0


In [257]:
test['predictions'] = preds_2
test

,Sequence,2_mer,4_mer,predictions
0,AATCTCTGTTGTTGTTATGCAAAAAAGGAATAATGTAATGTATTAT...,aa at tc ct tc ct tg gt tt tg gt tt tg gt tt t...,aatc atct tctc ctct tctg ctgt tgtt gttg ttgt t...,0.0
1,TTCTCTAAGAAAGTCGATGCTAAGCGGATGCTAAACGCAGGGCCGG...,tt tc ct tc ct ta aa ag ga aa aa ag gt tc cg g...,ttct tctc ctct tcta ctaa taag aaga agaa gaaa a...,1.0
2,ACATTTTACGGTCTTGCGTGTGTATTTATTGTTCGTACAAGTCAAA...,ac ca at tt tt tt ta ac cg gg gt tc ct tt tg g...,acat catt attt tttt ttta ttac tacg acgg cggt g...,0.0
3,CACTGAAAAAAAAAGAAAGGCTTATTTACTATTAACAACTTTAATA...,ca ac ct tg ga aa aa aa aa aa aa aa aa ag ga a...,cact actg ctga tgaa gaaa aaaa aaaa aaaa aaaa a...,0.0
4,GGGAGTGGCAACATGGGCTCACAAGTCTAGATCGACTCGGCTATTG...,gg gg ga ag gt tg gg gc ca aa ac ca at tg gg g...,ggga ggag gagt agtg gtgg tggc ggca gcaa caac a...,0.0
...,...,...,...,...
173,TAAATTGCATGTAAAATCATTAATAACGATTATATTCAGTTTGGCC...,ta aa aa at tt tg gc ca at tg gt ta aa aa aa a...,taaa aaat aatt attg ttgc tgca gcat catg atgt t...,1.0
174,GGATGTGCTTAGTGTAATTTGCTTATAAAAACTACTTTATATTTTC...,gg ga at tg gt tg gc ct tt ta ag gt tg gt ta a...,ggat gatg atgt tgtg gtgc tgct gctt ctta ttag t...,1.0
175,ATTCAGAAATTACATGTTTCTGAAAACTCTTTCCGATCAAAAACAA...,at tt tc ca ag ga aa aa at tt ta ac ca at tg g...,attc ttca tcag caga agaa gaaa aaat aatt atta t...,0.0
176,ACGGTTTTAAGTGCCCAAACTTAGGGTGTAGCGCCCTTTTCAGTTC...,ac cg gg gt tt tt tt ta aa ag gt tg gc cc cc c...,acgg cggt ggtt gttt tttt ttta ttaa taag aagt a...,0.0


# 4-mer take two

In [259]:
from tensorflow.keras.callbacks import EarlyStopping

In [264]:
X = train['4_mer']
y = train['is_promoter']

# Split train & test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tokenize and transform to integer index
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index
maxlen = max(len(x) for x in X_train) # longest text in train set

# Add pading to ensure all vectors have same dimensionality
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

embedding_dim = 200

model_4 = Sequential()
model_4.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
model_4.add(Conv1D(200, 5, activation='relu'))
model_4.add(MaxPooling1D(pool_size=2))
model_4.add(layers.Conv1D(200, 5, activation='relu'))
model_4.add(MaxPooling1D(pool_size=2))
model_4.add(Flatten())
model_4.add(Dense(128, activation = 'relu'))
model_4.add(Dense(32, activation = 'relu'))
model_4.add(Dense(10, activation='relu'))
model_4.add(Dense(1, activation='sigmoid'))
model_4.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

early_stop_combo = EarlyStopping(monitor='val_loss', patience=3, min_delta = 0.01, restore_best_weights=True)

print(model_4.summary())

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_19 (Embedding)    (None, 298, 200)          51400     
                                                                 
 conv1d_38 (Conv1D)          (None, 294, 200)          200200    
                                                                 
 max_pooling1d_38 (MaxPoolin  (None, 147, 200)         0         
 g1D)                                                            
                                                                 
 conv1d_39 (Conv1D)          (None, 143, 200)          200200    
                                                                 
 max_pooling1d_39 (MaxPoolin  (None, 71, 200)          0         
 g1D)                                                            
                                                                 
 flatten_19 (Flatten)        (None, 14200)           

In [265]:
history = model_4.fit(X_train,
                    y_train, 
                    batch_size=256,
                    validation_data=(X_test, y_test),
                    epochs=10,
                    callbacks=[early_stop_combo],
                    verbose=1)

Epoch 1/10
71/71 [==============================] - 148s 2s/step - loss: 0.5270 - accuracy: 0.7211 - val_loss: 0.4262 - val_accuracy: 0.8086
Epoch 2/10
71/71 [==============================] - 147s 2s/step - loss: 0.3478 - accuracy: 0.8479 - val_loss: 0.3198 - val_accuracy: 0.8688
Epoch 3/10
71/71 [==============================] - 174s 2s/step - loss: 0.2621 - accuracy: 0.8926 - val_loss: 0.2830 - val_accuracy: 0.8854
Epoch 4/10
71/71 [==============================] - 165s 2s/step - loss: 0.2269 - accuracy: 0.9095 - val_loss: 0.2849 - val_accuracy: 0.8887
Epoch 5/10
71/71 [==============================] - 149s 2s/step - loss: 0.1908 - accuracy: 0.9228 - val_loss: 0.2616 - val_accuracy: 0.9004
Epoch 6/10
71/71 [==============================] - 151s 2s/step - loss: 0.1504 - accuracy: 0.9433 - val_loss: 0.3001 - val_accuracy: 0.8916
Epoch 7/10
71/71 [==============================] - 151s 2s/step - loss: 0.1008 - accuracy: 0.9653 - val_loss: 0.2978 - val_accuracy: 0.8993
Epoch 8/10
71

In [266]:
X_test_final = test['4_mer']

In [267]:
X_test_final = tokenizer.texts_to_sequences(X_test_final)

maxlen = 298 # longest text in train set

# Add pading to ensure all vectors have same dimensionality
X_test_final = pad_sequences(X_test_final, padding='post', maxlen=maxlen)

In [268]:
preds_4 = model_4.predict(X_test_final)

In [269]:
preds_4

array([[3.33348513e-02],
       [9.88669097e-01],
       [1.46045536e-01],
       [1.35210156e-03],
       [1.15507245e-02],
       [4.46345270e-01],
       [9.87120628e-01],
       [9.66238976e-01],
       [1.93783283e-01],
       [9.95756984e-01],
       [9.98233497e-01],
       [7.70989060e-03],
       [3.13017368e-02],
       [6.12760425e-01],
       [6.30388081e-01],
       [9.88309145e-01],
       [2.51860023e-02],
       [1.24617219e-02],
       [1.63734078e-01],
       [9.66724396e-01],
       [8.49908590e-03],
       [5.39140999e-01],
       [9.93520141e-01],
       [1.52981579e-02],
       [3.78541946e-02],
       [9.92216468e-01],
       [9.82640505e-01],
       [9.92381930e-01],
       [9.69283104e-01],
       [6.64798439e-01],
       [2.90701687e-02],
       [6.72510266e-03],
       [4.30593818e-01],
       [2.48978287e-01],
       [9.97110426e-01],
       [2.18051672e-03],
       [1.96462274e-02],
       [5.29388785e-01],
       [4.88460660e-02],
       [2.07685083e-01],


In [270]:
labels=np.array(preds_4)
labels[labels>=0.5]=1
labels[labels<0.5]=0

In [271]:
preds_4 = pd.DataFrame(data = labels, columns = ['predictions_2'])
preds_4

,predictions_2
0,0.0
1,1.0
2,0.0
3,0.0
4,0.0
...,...
173,1.0
174,1.0
175,0.0
176,0.0


In [285]:
test['predictions'] = preds_4
test

,Sequence,predictions,predictions_2
0,AATCTCTGTTGTTGTTATGCAAAAAAGGAATAATGTAATGTATTAT...,0.0,0.0
1,TTCTCTAAGAAAGTCGATGCTAAGCGGATGCTAAACGCAGGGCCGG...,1.0,1.0
2,ACATTTTACGGTCTTGCGTGTGTATTTATTGTTCGTACAAGTCAAA...,0.0,0.0
3,CACTGAAAAAAAAAGAAAGGCTTATTTACTATTAACAACTTTAATA...,0.0,0.0
4,GGGAGTGGCAACATGGGCTCACAAGTCTAGATCGACTCGGCTATTG...,0.0,0.0
...,...,...,...
173,TAAATTGCATGTAAAATCATTAATAACGATTATATTCAGTTTGGCC...,1.0,1.0
174,GGATGTGCTTAGTGTAATTTGCTTATAAAAACTACTTTATATTTTC...,1.0,1.0
175,ATTCAGAAATTACATGTTTCTGAAAACTCTTTCCGATCAAAAACAA...,0.0,0.0
176,ACGGTTTTAAGTGCCCAAACTTAGGGTGTAGCGCCCTTTTCAGTTC...,0.0,0.0


In [286]:
test.drop(columns=['predictions_2'], inplace=True)
test.head()

,Sequence,predictions
0,AATCTCTGTTGTTGTTATGCAAAAAAGGAATAATGTAATGTATTAT...,0.0
1,TTCTCTAAGAAAGTCGATGCTAAGCGGATGCTAAACGCAGGGCCGG...,1.0
2,ACATTTTACGGTCTTGCGTGTGTATTTATTGTTCGTACAAGTCAAA...,0.0
3,CACTGAAAAAAAAAGAAAGGCTTATTTACTATTAACAACTTTAATA...,0.0
4,GGGAGTGGCAACATGGGCTCACAAGTCTAGATCGACTCGGCTATTG...,0.0


In [293]:
test

,Sequence,predictions
0,AATCTCTGTTGTTGTTATGCAAAAAAGGAATAATGTAATGTATTAT...,0.0
1,TTCTCTAAGAAAGTCGATGCTAAGCGGATGCTAAACGCAGGGCCGG...,1.0
2,ACATTTTACGGTCTTGCGTGTGTATTTATTGTTCGTACAAGTCAAA...,0.0
3,CACTGAAAAAAAAAGAAAGGCTTATTTACTATTAACAACTTTAATA...,0.0
4,GGGAGTGGCAACATGGGCTCACAAGTCTAGATCGACTCGGCTATTG...,0.0
...,...,...
173,TAAATTGCATGTAAAATCATTAATAACGATTATATTCAGTTTGGCC...,1.0
174,GGATGTGCTTAGTGTAATTTGCTTATAAAAACTACTTTATATTTTC...,1.0
175,ATTCAGAAATTACATGTTTCTGAAAACTCTTTCCGATCAAAAACAA...,0.0
176,ACGGTTTTAAGTGCCCAAACTTAGGGTGTAGCGCCCTTTTCAGTTC...,0.0


In [287]:
test.to_csv('../data/test_submission_2.csv')

In [277]:
test_2 = test.drop(columns=['predictions_2'], inplace=True)
test_3 = test.drop(columns=['predictions'], inplace=True)

In [280]:
test_2 = test

In [289]:
sub1 = pd.read_csv('../data/test_submission_1.csv')
sub2 = pd.read_csv('../data/test_submission_2.csv')

In [290]:
sub1['predictions'].equals(sub2['predictions'])

False

In [291]:
sub1['predictions'].sum()


78.0

In [292]:
sub2['predictions'].sum()

76.0

In [ ]:
done